# Taller 1 Conceptos básicos

In [1]:
!myenv\Scripts\activate


c:\Users\guill\Documents\GitHub\Intro-IA-Generativa-Taller-1-Conceptos-Basicos>() 


In [2]:
!pip install -r requirements.txt

## 1. Introducción a NLP

### 1.1 Ejercicio: Investigar 3 aplicaciones actuales de NLP

### 1.2 Ejercicio: Usar la biblioteca nltk o spaCy en Python para tokenizar un texto corto.

#### 1.2.1 Ejercicio con Nltk

##### 1.2.1.1 Intalación de NLTK

##### 1.2.1.2 Importar las librerías

In [4]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer


In [5]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[

True

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
nltk.download("omw-1.4")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

##### 1.2.1.3 Escribir un texto

In [9]:
nltk_text = "I am learning about tokenization!"

##### 1.2.1.3 Tokenizar por oraciones

In [10]:
sentences = sent_tokenize(nltk_text)
print(f"Sentance list: {sentences}.")

Sentance list: ['I am learning about tokenization!'].


##### 1.2.1.4 Tokenizar por palabras

In [11]:
words = word_tokenize(nltk_text)
print(f"Word list: {words}.")

Word list: ['I', 'am', 'learning', 'about', 'tokenization', '!'].


##### 1.2.1.5 Tokenizar por subpalabras

In [12]:
stemmer = PorterStemmer()
stemmer_subwords = [stemmer.stem(word) for word in words]
print(f"Subword list: {stemmer_subwords}.")

Subword list: ['i', 'am', 'learn', 'about', 'token', '!'].


In [13]:
lemmatizer = WordNetLemmatizer()
lemmatizer_subwords = [lemmatizer.lemmatize(word) for word in words]
print(f"Subword list: {lemmatizer_subwords}.")

Subword list: ['I', 'am', 'learning', 'about', 'tokenization', '!'].


##### 1.2.1.6 Tokenizar por caracteres

In [14]:
characters = list(nltk_text)
print(f"Character List: {characters}")

Character List: ['I', ' ', 'a', 'm', ' ', 'l', 'e', 'a', 'r', 'n', 'i', 'n', 'g', ' ', 'a', 'b', 'o', 'u', 't', ' ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', '!']


#### 1.2.2 Ejercicio con spaCy

##### 1.2.2.1 Intalación de NLTK

In [15]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 11.8 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.8 MB 12.2 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 11.0 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 11.0 MB/s eta 0:00:01
     -------------------------------- ------ 10.7/12.8 MB 10.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en_core_web_sm 3.8.0
    Uninstalling en_core_web_sm-3.8.0:
      Successfully uninstalled en_core_web_sm-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


##### 1.2.2.2 Importar las librerías

In [16]:
import spacy

In [17]:
nlp = spacy.load("en_core_web_sm")

##### 1.2.2.3 Escribir un texto

In [18]:
spacy_text = "I am learning how to do tokenization with spacy. It is awesome!"

In [19]:
doc = nlp(spacy_text)

In [20]:
for sent in doc.sents:
    print(sent.text)

I am learning how to do tokenization with spacy.
It is awesome!


In [21]:
words = []
for token in doc:
    words.append(token.text)
words

['I',
 'am',
 'learning',
 'how',
 'to',
 'do',
 'tokenization',
 'with',
 'spacy',
 '.',
 'It',
 'is',
 'awesome',
 '!']

In [22]:
characters = list(spacy_text)
characters

['I',
 ' ',
 'a',
 'm',
 ' ',
 'l',
 'e',
 'a',
 'r',
 'n',
 'i',
 'n',
 'g',
 ' ',
 'h',
 'o',
 'w',
 ' ',
 't',
 'o',
 ' ',
 'd',
 'o',
 ' ',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 ' ',
 'w',
 'i',
 't',
 'h',
 ' ',
 's',
 'p',
 'a',
 'c',
 'y',
 '.',
 ' ',
 'I',
 't',
 ' ',
 'i',
 's',
 ' ',
 'a',
 'w',
 'e',
 's',
 'o',
 'm',
 'e',
 '!']

## 2. Embeddings y Representaciones Vectoriales

### Ejercicio: Cargar un modelo de word embeddings de Hugging Face (por ejemplo, glove o word2vec) y visualizar los embeddings de palabras similares

### Ejercicio: Comparar GloVe (estático) con BERT (dinámico) mediante un ejemplo práctico en Python.

## 3. Modelos Basados en Redes Neuronales

### Ejercicio: Implementar una RNN simple para predicción de secuencias en Python.

### Ejercicio: Utilizar el modelo de transformers BERT o GPT-2 de Hugging Face para completar una frase simple.

In [35]:
from transformers import pipeline

In [36]:
text = "Being a computer scientist is all about [MASK]."

#### Utilizar BERT

In [31]:
bert_mask = pipeline("fill-mask", model="bert-base-uncased") 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
pred = bert_mask(text)
pred

[{'score': 0.11361566185951233,
  'token': 7588,
  'token_str': 'computers',
  'sequence': 'being a computer scientist is all about computers.'},
 {'score': 0.06099285930395126,
  'token': 2671,
  'token_str': 'science',
  'sequence': 'being a computer scientist is all about science.'},
 {'score': 0.05559048056602478,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'being a computer scientist is all about it.'},
 {'score': 0.048937588930130005,
  'token': 2470,
  'token_str': 'research',
  'sequence': 'being a computer scientist is all about research.'},
 {'score': 0.031744930893182755,
  'token': 8785,
  'token_str': 'math',
  'sequence': 'being a computer scientist is all about math.'}]

#### Utilizar GPT 2 TODO

In [38]:
gpt2_mask = pipeline("fill-mask", model="gpt2") 

In [40]:
gpt2_prediction = gpt2_mask(text)
gpt2_prediction

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': "Being a computer scientist is all about [MASK]. It's all about finding the perfect balance and finding the best solution. A lot of students may be a little surprised that some of them get so far [at a point] to realize that there"}]

## 4. Modelos Pre-entrenados y Fine-Tuning

### Pregunta 7: ¿Qué significa que un modelo esté preentrenado? ¿Cómo se puede ajustar para una tarea específica?

#### Ejercicio: Usar un modelo de Hugging Face (distilBERT) para clasificar un conjunto de datos de sentimientos (usar el dataset de Rotten tomatoes).

### Pregunta 8: ¿Qué es el enmascaramiento de palabras en el preentrenamiento de modelos como BERT?

#### Ejercicio: Usar BERT para predecir palabras enmascaradas en una oración

In [39]:
from transformers import pipeline

In [43]:
text = "Being a computer scientist is all about [MASK]."

In [44]:
bert_mask = pipeline("fill-mask", model="bert-base-uncased") 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
pred = bert_mask(text)
pred

[{'score': 0.11361566185951233,
  'token': 7588,
  'token_str': 'computers',
  'sequence': 'being a computer scientist is all about computers.'},
 {'score': 0.06099285930395126,
  'token': 2671,
  'token_str': 'science',
  'sequence': 'being a computer scientist is all about science.'},
 {'score': 0.05559048056602478,
  'token': 2009,
  'token_str': 'it',
  'sequence': 'being a computer scientist is all about it.'},
 {'score': 0.048937588930130005,
  'token': 2470,
  'token_str': 'research',
  'sequence': 'being a computer scientist is all about research.'},
 {'score': 0.031744930893182755,
  'token': 8785,
  'token_str': 'math',
  'sequence': 'being a computer scientist is all about math.'}]

## 5. Tareas Clásicas de NLP

### Pregunta 9: ¿Qué es la clasificación de texto y qué modelos se suelen utilizar para esta tarea?

#### Ejercicio: Implementar un modelo NER utilizando spaCy o Hugging Face y aplicar etiquetas a un conjunto de texto.

### Pregunta 10: ¿Qué es el análisis de sentimientos y cómo se relaciona con la clasificación de texto?

#### Ejercicio: Utilizar Hugging Face para hacer análisis de sentimientos en un conjunto de datos de reseñas (usar el dataset de IMDB).

## 6. Generación de Texto

### Ejercicio: Usar Hugging Face para generar texto a partir de un prompt con GPT-2.

### Ejercicio: Discutir sobre sesgos en los modelos de lenguaje generativo y su impacto en aplicaciones reales.

## 7. Evaluación de Modelos NLP

### Ejercicio: Usar métricas como accuracy, precision, y recall para evaluar un modelo de clasificación de texto.

### Ejercicio: Calcular la perplejidad de un modelo de lenguaje en Hugging Face para una tarea de generación de texto.